In [3]:
query_folder = '/home/vineeth/Downloads/Search Cattle Problem Statement/Images/Test/'
image_folder = '/home/vineeth/Downloads/Search Cattle Problem Statement/Images/Database/'

In [4]:
import cv2
import os

def template_matching(query_folder, images_folder):
    # List to store the similarity scores
    matches = []

    # Iterate over the images in the query folder
    for query_filename in os.listdir(query_folder):
        if query_filename.endswith('.jpg') or query_filename.endswith('.png'):
            # Load the query image from the query folder and convert to grayscale
            query_path = os.path.join(query_folder, query_filename)
            query_gray = cv2.imread(query_path, 0)

            # List to store the similarity scores for the current query image
            scores = []

            # Iterate over the images in the target folder
            for image_filename in os.listdir(images_folder):
                if image_filename.endswith('.jpg') or image_filename.endswith('.png'):
                    # Load the target image from the target folder and convert to grayscale
                    image_path = os.path.join(images_folder, image_filename)
                    image_gray = cv2.imread(image_path, 0)

                    # Resize the template image to match the size of the target image
                    resized_template = cv2.resize(query_gray, (image_gray.shape[1], image_gray.shape[0]))

                    # Perform template matching
                    result = cv2.matchTemplate(image_gray, resized_template, cv2.TM_CCOEFF_NORMED)

                    # Get the maximum correlation value from the result
                    similarity = cv2.minMaxLoc(result)[1]

                    # Append the similarity score to the list
                    scores.append((image_filename, similarity))

            # Sort the scores based on the similarity in descending order
            scores.sort(key=lambda x: x[1], reverse=True)

            # Check if any match is found
            if scores:
                # Add the best match to the matches list
                best_match = {"query_image": query_filename, "match_image": scores[0][0], "similarity": scores[0][1]}
                matches.append(best_match)
                
            for ele in matches:
                if ele['similarity'] > 0.5 :
                    pass
                else:
                    ele['match_image'] = 'Not Found'

    return matches


In [5]:
template_matching(query_folder,image_folder)

[{'query_image': 'test_cattle_3.jpg',
  'match_image': 'database_cattle_7.jpg',
  'similarity': 0.6066282391548157},
 {'query_image': 'test_cattle_2.jpg',
  'match_image': 'Not Found',
  'similarity': 0.44811251759529114},
 {'query_image': 'test_cattle_5.jpg',
  'match_image': 'database_cattle_20.jpg',
  'similarity': 0.6859709024429321},
 {'query_image': 'test_cattle_4.jpg',
  'match_image': 'database_cattle_13.jpg',
  'similarity': 0.5071526765823364},
 {'query_image': 'test_cattle_1.jpg',
  'match_image': 'database_cattle_18.jpg',
  'similarity': 0.5615406632423401}]

In [6]:
import cv2
import os

def histogram_comparison(query_folder, images_folder):
    # List to store the similarity scores
    matches = []

    # Iterate over the images in the query folder
    for query_filename in os.listdir(query_folder):
        if query_filename.endswith('.jpg') or query_filename.endswith('.png'):
            # Load the query image from the query folder and convert to grayscale
            query_path = os.path.join(query_folder, query_filename)
            query_gray = cv2.imread(query_path, 0)

            # Calculate histogram of the query image
            query_hist = cv2.calcHist([query_gray], [0], None, [256], [0, 256])
            query_hist = cv2.normalize(query_hist, query_hist).flatten()

            # List to store the similarity scores for the current query image
            scores = []

            # Iterate over the images in the target folder
            for image_filename in os.listdir(images_folder):
                if image_filename.endswith('.jpg') or image_filename.endswith('.png'):
                    # Load the target image from the target folder and convert to grayscale
                    image_path = os.path.join(images_folder, image_filename)
                    image_gray = cv2.imread(image_path, 0)

                    # Calculate histogram of the current target image
                    image_hist = cv2.calcHist([image_gray], [0], None, [256], [0, 256])
                    image_hist = cv2.normalize(image_hist, image_hist).flatten()

                    # Calculate histogram correlation as the similarity score
                    similarity = cv2.compareHist(query_hist, image_hist, cv2.HISTCMP_CORREL)

                    # Append the similarity score to the list
                    scores.append((image_filename, similarity))

            # Sort the scores based on the similarity in descending order
            scores.sort(key=lambda x: x[1], reverse=True)

            # Check if any match is found
            if scores:
                # Add the best match to the matches list
                best_match = {"query_image": query_filename, "match_image": scores[0][0], "similarity": scores[0][1]}
                matches.append(best_match)

            for ele in matches:
                if ele['similarity'] > 0.90 :
                    pass
                else:
                    ele['match_image'] = 'Not Found'

    return matches


In [7]:
histogram_comparison(query_folder,image_folder)

[{'query_image': 'test_cattle_3.jpg',
  'match_image': 'database_cattle_2.jpg',
  'similarity': 0.9553076677229945},
 {'query_image': 'test_cattle_2.jpg',
  'match_image': 'database_cattle_3.jpg',
  'similarity': 0.9163162707072561},
 {'query_image': 'test_cattle_5.jpg',
  'match_image': 'database_cattle_8.jpg',
  'similarity': 0.9159944470168122},
 {'query_image': 'test_cattle_4.jpg',
  'match_image': 'Not Found',
  'similarity': 0.8546108984848734},
 {'query_image': 'test_cattle_1.jpg',
  'match_image': 'database_cattle_11.jpg',
  'similarity': 0.9651266331253783}]

In [10]:
import cv2
import os

def feature_matching(query_folder, images_folder):
    # Create feature detector and descriptor objects
    sift = cv2.SIFT_create()

    # List to store the similarity scores
    matches_final = []

    # Iterate over the images in the query folder
    for query_filename in os.listdir(query_folder):
        if query_filename.endswith('.jpg') or query_filename.endswith('.png'):
            # Load the query image from the query folder and convert to grayscale
            query_path = os.path.join(query_folder, query_filename)
            query_gray = cv2.imread(query_path, 0)

            # Detect and compute keypoints and descriptors for the query image
            query_keypoints, query_descriptors = sift.detectAndCompute(query_gray, None)

            # List to store the similarity scores for the current query image
            scores = []

            # Iterate over the images in the target folder
            for image_filename in os.listdir(images_folder):
                if image_filename.endswith('.jpg') or image_filename.endswith('.png'):
                    # Load the target image from the target folder and convert to grayscale
                    image_path = os.path.join(images_folder, image_filename)
                    image_gray = cv2.imread(image_path, 0)

                    # Detect and compute keypoints and descriptors for the current target image
                    image_keypoints, image_descriptors = sift.detectAndCompute(image_gray, None)

                    # Create a matcher object
                    matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_FLANNBASED)

                    # Match the descriptors using the matcher
                    matches = matcher.knnMatch(query_descriptors, image_descriptors, k=2)

                    # Apply ratio test to filter good matches
                    good_matches = []
                    for m, n in matches:
                        if m.distance < 0.9 * n.distance:
                            good_matches.append(m)

                    # Calculate the similarity score based on the number of good matches
                    similarity = len(good_matches) / max(len(query_keypoints), len(image_keypoints))

                    # Append the similarity score to the list
                    scores.append((image_filename, similarity))

            # Sort the scores based on the similarity in descending order
            scores.sort(key=lambda x: x[1], reverse=True)

            # Check if any match is found
            if scores:
                # Add the best match to the matches list
                best_match = {"query_image": query_filename, "match_image": scores[0][0], "similarity": scores[0][1]}
                matches_final.append(best_match)
                
            for ele in matches_final:
                if ele['similarity'] > 0.2 :
                    pass
                else:
                    ele['match_image'] = 'Not Found'

    return matches_final


In [11]:
feature_matching(query_folder,image_folder)

[{'query_image': 'test_cattle_3.jpg',
  'match_image': 'database_cattle_20.jpg',
  'similarity': 0.21418539325842698},
 {'query_image': 'test_cattle_2.jpg',
  'match_image': 'Not Found',
  'similarity': 0.18665158371040724},
 {'query_image': 'test_cattle_5.jpg',
  'match_image': 'Not Found',
  'similarity': 0.19865319865319866},
 {'query_image': 'test_cattle_4.jpg',
  'match_image': 'Not Found',
  'similarity': 0.18243243243243243},
 {'query_image': 'test_cattle_1.jpg',
  'match_image': 'Not Found',
  'similarity': 0.1917808219178082}]